In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line8.csv')
data
# 카페 데이터만
data = data.loc[data['상권업종중분류명'] == '커피점/카페']  
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
3,61,22791439,로얄커피숍,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,134030,5385.0,NaN,1,NaN,127.124193,37.534901,8호선 천호(풍납토성),258.969474,NaN
7,117,20492749,알뮤트1917,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,134779,5328.0,NaN,5,NaN,127.124418,37.538849,8호선 천호(풍납토성),79.487388,NaN
11,164,20815616,카페클럽,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138200,5805.0,NaN,1,NaN,127.122480,37.488907,8호선 문정,157.974984,NaN
12,204,23163017,뽕뜨락카페,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,134050,5242.0,NaN,2,NaN,127.126922,37.549293,8호선 암사,186.636140,NaN
15,253,26463173,파란달의커피볶는집,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138160,5719.0,NaN,1,NaN,127.119651,37.493842,8호선 가락시장,188.015150,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,135210,17253870,카페노티드,잠실점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138849,5621.0,NaN,6,NaN,127.110628,37.509038,8호선 몽촌토성(평화의문),528.597913,NaN
6142,135211,17250152,스터디앤라이브러리,석촌센터,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138847,5697.0,NaN,NaN,NaN,127.108699,37.501005,8호선 송파,402.920947,NaN
6150,135406,17249266,카페블라썸,롯데백화점잠실점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138721,5554.0,NaN,11,NaN,127.100801,37.511649,8호선 잠실(송파구청),438.082061,NaN
6151,135407,17250101,저스트케이팝,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138934,5551.0,NaN,11,NaN,127.104284,37.513631,8호선 잠실(송파구청),89.190797,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
3,로얄커피숍,커피점/카페,커피전문점/카페/다방,서울특별시 강동구 천호옛길 65,8호선 천호(풍납토성),258.969474,NaN,천호,천호역 로얄커피숍
7,알뮤트1917,커피점/카페,커피전문점/카페/다방,서울특별시 강동구 천호대로 1005,8호선 천호(풍납토성),79.487388,NaN,천호,천호역 알뮤트1917
11,카페클럽,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 동남로2길 3-6,8호선 문정,157.974984,NaN,문정,문정역 카페클럽
12,뽕뜨락카페,커피점/카페,커피전문점/카페/다방,서울특별시 강동구 올림픽로 763,8호선 암사,186.636140,NaN,암사,암사역 뽕뜨락카페
15,파란달의커피볶는집,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 송파대로28길 16,8호선 가락시장,188.015150,NaN,가락시장,가락시장역 파란달의커피볶는집
...,...,...,...,...,...,...,...,...,...
6141,카페노티드,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 백제고분로45길 3,8호선 몽촌토성(평화의문),528.597913,NaN,몽촌토성,몽촌토성역 카페노티드
6142,스터디앤라이브러리,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 가락로16길 3-4,8호선 송파,402.920947,NaN,송파,송파역 스터디앤라이브러리
6150,카페블라썸,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 송파대로 521,8호선 잠실(송파구청),438.082061,NaN,잠실,잠실역 카페블라썸
6151,저스트케이팝,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 올림픽로 300,8호선 잠실(송파구청),89.190797,NaN,잠실,잠실역 저스트케이팝


In [4]:
items = data["검색어"]
items

3             천호역 로얄커피숍
7           천호역 알뮤트1917
11             문정역 카페클럽
12            암사역 뽕뜨락카페
15      가락시장역 파란달의커피볶는집
             ...       
6141        몽촌토성역 카페노티드
6142      송파역 스터디앤라이브러리
6150          잠실역 카페블라썸
6151         잠실역 저스트케이팝
6169         문정역 세라젬웰카페
Name: 검색어, Length: 911, dtype: object

In [5]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [6]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/911
식당 존재
https://place.map.kakao.com/20602043
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2/911
식당 존재 x
진행상황 : 3/911
식당 존재
https://place.map.kakao.com/20181564
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/911
식당 존재 x
진행상황 : 5/911
식당 존재
https://place.map.kakao.com/23482678
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 6/911
식당 존재
https://place.map.kakao.com/1833008632
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/911
식당 존재 x
진행상황 : 8/911
식당 존재 x
진행상황 : 9/911
식당 존재
https://place.map.kakao.com/27515201
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 10/911
식당 존재 x
진행상황 : 11/911
식당 존재 x
진행상황 : 12/911
식당 존재
https://place.map.kakao.com/1277331258
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/911
식당 존재 x
진행상황 : 14/911
식당 존재
https://place.map.kakao.com/735973978
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 15/911
식당 존재
https://place.map.kakao.com/710586475
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 16/911
식당 존재 x
진행상황 : 17/911
식당 존재
https://place.map.kakao.com/8099936
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 18/911
식당 존재 x
진행상황 

0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 141/911
식당 존재 x
진행상황 : 142/911
식당 존재
https://place.map.kakao.com/1246863420
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 143/911
식당 존재
https://place.map.kakao.com/1312834698
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 144/911
식당 존재
https://place.map.kakao.com/244820280
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 145/911
식당 존재
https://place.map.kakao.com/1603981505
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 146/911
식당 존재
https://place.map.kakao.com/27250770
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 147/911
식당 존재
https://place.map.kakao.com/21326288
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 148/911
식당 존재
https://place.map.kakao.com/1770743183
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 149/911
식당 존재
https://place.map.kakao.com/306261602
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 150/911
식당 존재
https://place.map.kakao.com/1859218342
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 151/911
식당 존재
https://place.map.kakao.com/1283611426
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 152/911
식당 존재 x
진행상황 : 153/911
식

식당 존재 x
진행상황 : 314/911
식당 존재
https://place.map.kakao.com/645708014
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 315/911
식당 존재 x
진행상황 : 316/911
식당 존재
https://place.map.kakao.com/23109066
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 317/911
식당 존재 x
진행상황 : 318/911
식당 존재
https://place.map.kakao.com/1856257072
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 319/911
식당 존재 x
진행상황 : 320/911
식당 존재
https://place.map.kakao.com/697234117
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 321/911
식당 존재
https://place.map.kakao.com/953709771
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 322/911
식당 존재
https://place.map.kakao.com/26239301
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 323/911
식당 존재
https://place.map.kakao.com/488907063
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 324/911
식당 존재
https://place.map.kakao.com/20748298
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 325/911
식당 존재 x
진행상황 : 326/911
식당 존재 x
진행상황 : 327/911
식당 존재
https://place.map.kakao.com/1931377665
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 328/911
식당 존재
https://place.map.kakao.com/1669692931
1
더 이상 리

식당 존재 x
진행상황 : 428/911
식당 존재
https://place.map.kakao.com/262466070
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 429/911
식당 존재
https://place.map.kakao.com/1319161148
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 430/911
식당 존재
https://place.map.kakao.com/1628719994
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 431/911
식당 존재
https://place.map.kakao.com/1360779553
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 432/911
식당 존재
https://place.map.kakao.com/472671966
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 433/911
식당 존재 x
진행상황 : 434/911
식당 존재
https://place.map.kakao.com/1551189745
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 435/911
식당 존재 x
진행상황 : 436/911
식당 존재
https://place.map.kakao.com/199323440
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 437/911
식당 존재
https://place.map.kakao.com/220137314
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 438/911
식당 존재
https://place.map.kakao.com/1884470166
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 439/911
식당 존재 x
진행상황 : 440/911
식당 존재
https://place.map.kakao.com/19037010
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 441/911
식당 존재 x
진행상황 : 442/911
식당

더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 552/911
식당 존재
https://place.map.kakao.com/740702480
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 553/911
식당 존재
https://place.map.kakao.com/2122753400
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 554/911
식당 존재
https://place.map.kakao.com/999800504
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 555/911
식당 존재
https://place.map.kakao.com/1785002085
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 556/911
식당 존재 x
진행상황 : 557/911
식당 존재
https://place.map.kakao.com/397685200
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 558/911
식당 존재 x
진행상황 : 559/911
식당 존재
https://place.map.kakao.com/169748069
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 560/911
식당 존재 x
진행상황 : 561/911
식당 존재 x
진행상황 : 562/911
식당 존재
https://place.map.kakao.com/27220883
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 563/911
식당 존재
https://place.map.kakao.com/2141463273
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 564/911
식당 존재 x
진행상황 : 565/911
식당 존재 x
진행상황 : 566/911
식당 존재 x
진행상황 : 567/911
식당 존재
https://place.map.kakao.com/172

식당 존재
https://place.map.kakao.com/710586475
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 673/911
식당 존재 x
진행상황 : 674/911
식당 존재
https://place.map.kakao.com/898111174
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 675/911
식당 존재
https://place.map.kakao.com/581669019
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 676/911
식당 존재
https://place.map.kakao.com/715346014
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 677/911
식당 존재
https://place.map.kakao.com/26022618
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 678/911
식당 존재
https://place.map.kakao.com/1155940414
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 679/911
식당 존재
https://place.map.kakao.com/1088083452
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 680/911
식당 존재
https://place.map.kakao.com/1924672079
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 681/911
식당 존재
https://place.map.kakao.com/1700106518
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 682/911
식당 존재
https://place.map.kakao.com/421470142
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 683/911
식당 존재
https://place.map.kakao.com/1424942284
5
더 이상 리뷰 존재 X
기본 페이지로 

식당 존재 x
진행상황 : 794/911
식당 존재
https://place.map.kakao.com/900095185
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 795/911
식당 존재 x
진행상황 : 796/911
식당 존재 x
진행상황 : 797/911
식당 존재
https://place.map.kakao.com/853049870
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 798/911
식당 존재 x
진행상황 : 799/911
식당 존재 x
진행상황 : 800/911
식당 존재
https://place.map.kakao.com/8143024
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 801/911
식당 존재
https://place.map.kakao.com/856712127
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 802/911
식당 존재
https://place.map.kakao.com/74048772
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 803/911
식당 존재 x
진행상황 : 804/911
식당 존재
https://place.map.kakao.com/255416162
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 805/911
식당 존재 x
진행상황 : 806/911
식당 존재
https://place.map.kakao.com/2044440392
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 807/911
식당 존재
https://place.map.kakao.com/1742158853
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 808/911
식당 존재 x
진행상황 : 809/911
식당 존재
https://place.map.kakao.com/248966980
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 810/911
식당 존재
https://place.map.k

In [7]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,7nom52,문정역 카페클럽,5점,2019.06.13.
1,None,문정역 카페클럽,None,2017.01.11.
2,,문정역 카페클럽,5점,2013.07.03.
3,10017dq,가락시장역 파란달의커피볶는집,4점,2019.06.27.
4,7uqu8c,가락시장역 파란달의커피볶는집,3점,2018.08.10.
...,...,...,...,...
2890,10d3d90,잠실역 저스트케이팝,1점,2019.05.19.
2891,uvbngm,잠실역 저스트케이팝,5점,2019.03.07.
2892,uva8ip,잠실역 저스트케이팝,5점,2019.03.07.
2893,lnmpgd,잠실역 저스트케이팝,5점,2019.01.24.


In [8]:
img_elem

,ItemID,img_url
0,문정역 카페클럽,None
1,문정역 카페클럽,None
2,문정역 카페클럽,None
3,가락시장역 파란달의커피볶는집,None
4,가락시장역 파란달의커피볶는집,None
...,...,...
2890,잠실역 저스트케이팝,None
2891,잠실역 저스트케이팝,[]
2892,잠실역 저스트케이팝,[]
2893,잠실역 저스트케이팝,[]


In [9]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
3,로얄커피숍,커피점/카페,커피전문점/카페/다방,서울특별시 강동구 천호옛길 65,8호선 천호(풍납토성),258.969474,https://place.map.kakao.com/20602043,천호,천호역 로얄커피숍
7,알뮤트1917,커피점/카페,커피전문점/카페/다방,서울특별시 강동구 천호대로 1005,8호선 천호(풍납토성),79.487388,None,천호,천호역 알뮤트1917
11,카페클럽,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 동남로2길 3-6,8호선 문정,157.974984,https://place.map.kakao.com/20181564,문정,문정역 카페클럽
12,뽕뜨락카페,커피점/카페,커피전문점/카페/다방,서울특별시 강동구 올림픽로 763,8호선 암사,186.636140,None,암사,암사역 뽕뜨락카페
15,파란달의커피볶는집,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 송파대로28길 16,8호선 가락시장,188.015150,https://place.map.kakao.com/23482678,가락시장,가락시장역 파란달의커피볶는집
...,...,...,...,...,...,...,...,...,...
6141,카페노티드,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 백제고분로45길 3,8호선 몽촌토성(평화의문),528.597913,https://place.map.kakao.com/1215442657,몽촌토성,몽촌토성역 카페노티드
6142,스터디앤라이브러리,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 가락로16길 3-4,8호선 송파,402.920947,https://place.map.kakao.com/394710071,송파,송파역 스터디앤라이브러리
6150,카페블라썸,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 송파대로 521,8호선 잠실(송파구청),438.082061,https://place.map.kakao.com/874421812,잠실,잠실역 카페블라썸
6151,저스트케이팝,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 올림픽로 300,8호선 잠실(송파구청),89.190797,https://place.map.kakao.com/2132355576,잠실,잠실역 저스트케이팝


In [10]:
img_elem

,ItemID,img_url
0,문정역 카페클럽,None
1,문정역 카페클럽,None
2,문정역 카페클럽,None
3,가락시장역 파란달의커피볶는집,None
4,가락시장역 파란달의커피볶는집,None
...,...,...
2890,잠실역 저스트케이팝,None
2891,잠실역 저스트케이팝,[]
2892,잠실역 저스트케이팝,[]
2893,잠실역 저스트케이팝,[]


In [11]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,7nom52,문정역 카페클럽,5점,2019.06.13.
1,None,문정역 카페클럽,None,2017.01.11.
2,,문정역 카페클럽,5점,2013.07.03.
3,10017dq,가락시장역 파란달의커피볶는집,4점,2019.06.27.
4,7uqu8c,가락시장역 파란달의커피볶는집,3점,2018.08.10.
...,...,...,...,...
2890,10d3d90,잠실역 저스트케이팝,1점,2019.05.19.
2891,uvbngm,잠실역 저스트케이팝,5점,2019.03.07.
2892,uva8ip,잠실역 저스트케이팝,5점,2019.03.07.
2893,lnmpgd,잠실역 저스트케이팝,5점,2019.01.24.


In [12]:
review_elem

,ItemID,review
0,문정역 카페클럽,친전하고 맛있어요~ 가격도착하고요
1,문정역 카페클럽,보통이에요
2,문정역 카페클럽,새로 생긴 커피점 인데 커피맛이 아주좋아요 추천함니다.
3,가락시장역 파란달의커피볶는집,저는 라떼보다 원두의 맛이 더 잘 느껴지고 진한 플랫화이트를 선호합니다. 메뉴에 ...
4,가락시장역 파란달의커피볶는집,보통이에요
...,...,...
2890,잠실역 저스트케이팝,일단 하드락 카페 배껴서 하는듯 하다 그러나 맛 공연 서비스 무엇하나 장점 없음 그...
2891,잠실역 저스트케이팝,롯데 월드 몰 맛집 저스트케이팝 선촌호수가 보이는 뷰도 너무 좋고 부담없이 브런...
2892,잠실역 저스트케이팝,Food is great and i get to enjoy a good view w...
2893,잠실역 저스트케이팝,굿굿 또갈거에요


In [13]:
img_elem.head()

,ItemID,img_url
0,문정역 카페클럽,None
1,문정역 카페클럽,None
2,문정역 카페클럽,None
3,가락시장역 파란달의커피볶는집,None
4,가락시장역 파란달의커피볶는집,None


In [14]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




       UserID      ItemID Rating    Timestamp
2884   7ki7fo  잠실역 저스트케이팝     1점  2021.01.17.
2885  1bre9hm  잠실역 저스트케이팝     4점  2021.01.17.
2886   7oqfvl  잠실역 저스트케이팝     5점  2020.12.03.
2887   ffaulk  잠실역 저스트케이팝     5점  2020.10.29.
2888   upmfe9  잠실역 저스트케이팝     4점  2020.08.12.
2889   fd0v4k  잠실역 저스트케이팝     1점  2019.07.27.
2890  10d3d90  잠실역 저스트케이팝     1점  2019.05.19.
2891   uvbngm  잠실역 저스트케이팝     5점  2019.03.07.
2892   uva8ip  잠실역 저스트케이팝     5점  2019.03.07.
2893   lnmpgd  잠실역 저스트케이팝     5점  2019.01.24.
2894   fkiun6  잠실역 저스트케이팝     5점  2019.01.21.
          ItemID img_url
2884  잠실역 저스트케이팝    None
2885  잠실역 저스트케이팝    None
2886  잠실역 저스트케이팝    None
2887  잠실역 저스트케이팝    None
2888  잠실역 저스트케이팝    None
2889  잠실역 저스트케이팝    None
2890  잠실역 저스트케이팝    None
2891  잠실역 저스트케이팝      []
2892  잠실역 저스트케이팝      []
2893  잠실역 저스트케이팝      []
2894  잠실역 저스트케이팝    None
          ItemID                                             review
2884  잠실역 저스트케이팝                                            재방문의사없음
2885  잠실역 저스

In [15]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line8_cafe(final).csv')

In [16]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe_User(final).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe_img(final).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lline5_cafe_review(final).csv')

In [17]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


348

In [18]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line8(final)url.csv')